In [1]:
import geopandas as gpd
import folium
import numpy as np
import os
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection

# Definir la ruta base de los datos
BASE_PATH = os.path.expanduser("~/Documents/CoyoacanDataAnalysis/data/")

In [2]:
def load_shapefile(file_path, rename_columns=None):
    """
    Carga un shapefile y opcionalmente renombra columnas.
    """
    gdf = gpd.read_file(file_path)
    if rename_columns:
        gdf = gdf.rename(columns=rename_columns)
    print(f"\nDatos cargados desde {file_path}:")
    print(gdf.head())
    return gdf

def filter_coyoacan(gdf, alcaldias_shp):
    """
    Filtra el GeoDataFrame para mantener solo los datos de Coyoacán.
    """
    gdf_alcaldias = gpd.read_file(alcaldias_shp)
    print("\nDatos de alcaldías cargados:")
    print(gdf_alcaldias[['NOMGEO', 'geometry']].head())

    gdf_coyoacan = gdf_alcaldias[gdf_alcaldias['NOMGEO'] == 'Coyoacán']
    print("\nGeometría de Coyoacán:")
    print(gdf_coyoacan)

    # Asegurar que ambos GeoDataFrames tengan el mismo CRS
    if gdf.crs != gdf_coyoacan.crs:
        gdf_coyoacan = gdf_coyoacan.to_crs(gdf.crs)

    gdf_clipped = gpd.clip(gdf, gdf_coyoacan)
    print("\nDatos después de filtrar para Coyoacán:")
    print(gdf_clipped.head())

    return gdf_clipped

def calculate_area_density(gdf, population_column, density_column):
    """
    Calcula el área en km² y la densidad de población.
    """
    gdf = gdf.to_crs(epsg=32614)  # UTM Zona 14N
    gdf['area_km2'] = gdf.geometry.area / 10**6  # Convertir m² a km²
    gdf['area_km2'] = gdf['area_km2'].replace(0, np.nan)  # Reemplazar áreas 0 con NaN
    gdf[density_column] = gdf[population_column] / gdf['area_km2']
    gdf[density_column] = gdf[density_column].fillna(0)

    print(f"\nEstadísticas de {density_column}:")
    print(gdf[density_column].describe())

    return gdf

def to_polygon(geometry):
    """
    Convierte geometrías complejas a Polygon.
    """
    try:
        if isinstance(geometry, Polygon):
            return geometry
        elif isinstance(geometry, MultiPolygon):
            return max(geometry, key=lambda a: a.area)  # Tomar el mayor polígono
        elif isinstance(geometry, GeometryCollection):
            polygons = [geom for geom in geometry.geoms if isinstance(geom, Polygon)]
            if polygons:
                return max(polygons, key=lambda a: a.area)
            else:
                print("Advertencia: GeometryCollection sin polígonos, se omitirá.")
                return None
        else:
            return None
    except Exception as e:
        print(f"Error procesando geometría: {e}")
        return None

def clean_geometries(gdf):
    """
    Limpia y convierte geometrías no soportadas a Polygon.
    """
    gdf = gdf[gdf.geometry.type.isin(['Polygon', 
                                      'MultiPolygon', 
                                      'GeometryCollection'])].copy()
    gdf['geometry'] = gdf['geometry'].apply(to_polygon)
    gdf = gdf[~gdf['geometry'].isnull()].copy()
    # Reparar geometrías inválidas
    gdf['geometry'] = gdf['geometry'].buffer(0)
    # Eliminar geometrías vacías
    gdf = gdf[~gdf['geometry'].is_empty]
    print("\nGeometrías después de limpiar:")
    print(gdf.geometry.head())
    return gdf

def create_folium_map(gdf, density_column, legend_name, fill_color, population_column, max_density):
    """
    Crea un mapa de Folium con una capa coroplética y líneas de subáreas en negro.
    """
    gdf = gdf.to_crs(epsg=4326)  # Reproyectar a WGS84

    # Calcular centroides en CRS proyectado y transformar a WGS84
    gdf_centroids = gdf.to_crs(epsg=32614).geometry.centroid.to_crs(epsg=4326)
    centro_lat = gdf_centroids.y.mean()
    centro_lon = gdf_centroids.x.mean()

    m = folium.Map(location=[centro_lat, centro_lon], zoom_start=13)

    # Crear capa coroplética con max_value y líneas en negro
    folium.Choropleth(
        geo_data=gdf,
        data=gdf,
        columns=['ageb', density_column],
        key_on='feature.properties.ageb',
        fill_color=fill_color,
        fill_opacity=0.7,
        line_opacity=0.5,
        line_color='black',  # Cambiar el color de las líneas a negro
        legend_name=legend_name,
        max_value=max_density
    ).add_to(m)

    # Añadir tooltips con información adicional y líneas en negro
    tooltip = folium.GeoJsonTooltip(
        fields=['ageb', population_column, density_column],
        aliases=['AGEB:', 'Población:', legend_name],
        localize=True
    )

    folium.GeoJson(
        gdf,
        tooltip=tooltip,
        style_function=lambda feature: {
            'fillColor': 'transparent',
            'color': 'black',  # Cambiar el color de las líneas a negro
            'weight': 0.5
        }
    ).add_to(m)

    return m

In [3]:
# Paso 1: Cargar datos demográficos
print("Cargando datos demográficos...")
demografico_path = os.path.join(BASE_PATH, "demografico/2020")
hombres_shp = os.path.join(demografico_path, "hombres.shp")
mujeres_shp = os.path.join(demografico_path, "mujeres.shp")
total_shp = os.path.join(demografico_path, "total.shp")

gdf_total = load_shapefile(total_shp, rename_columns={'pob': 'pob_total'})
gdf_hombres = load_shapefile(hombres_shp, rename_columns={'pob': 'pob_hombres'})
gdf_mujeres = load_shapefile(mujeres_shp, rename_columns={'pob': 'pob_mujeres'})

# Paso 2: Unir datos de población por género al total usando 'ageb' como clave
print("\nUniendo datos de población...")
print("Total de registros antes de unir:", len(gdf_total))
gdf_total = gdf_total.merge(gdf_hombres[['ageb', 'pob_hombres']], on='ageb', how='left')
gdf_total = gdf_total.merge(gdf_mujeres[['ageb', 'pob_mujeres']], on='ageb', how='left')
print("Total de registros después de unir:", len(gdf_total))
print("\nDatos después de la unión:")
print(gdf_total[['ageb', 'pob_total', 'pob_hombres', 'pob_mujeres']].head())

# Paso 3: Manejo de valores nulos
print("\nManejando valores nulos...")
gdf_total.dropna(subset=['pob_total', 'pob_hombres', 'pob_mujeres'], inplace=True)
# Convertir a números enteros
gdf_total[['pob_total', 'pob_hombres', 'pob_mujeres']] = gdf_total[['pob_total', 'pob_hombres', 'pob_mujeres']].astype(int)

# Paso 4: Verificación de consistencia en poblaciones
print("\nVerificando consistencia en los datos de población...")
gdf_total['pob_total_calc'] = gdf_total['pob_hombres'] + gdf_total['pob_mujeres']
inconsistentes = gdf_total[gdf_total['pob_total'] != gdf_total['pob_total_calc']]
print(f"Registros con inconsistencias en población: {len(inconsistentes)}")
# Ajustar pob_total si hay inconsistencias menores
gdf_total.loc[gdf_total['pob_total'] != gdf_total['pob_total_calc'], 'pob_total'] = gdf_total['pob_total_calc']
gdf_total.drop(columns=['pob_total_calc'], inplace=True)

# Paso 5: Eliminar duplicados
print("\nEliminando duplicados...")
gdf_total.drop_duplicates(subset='ageb', inplace=True)
print(f"Total de registros después de eliminar duplicados: {len(gdf_total)}")

# Paso 6: Filtrar datos para Coyoacán
print("\nFiltrando datos para Coyoacán...")
alcaldias_shp = os.path.join(BASE_PATH, "limites/poligonos_alcaldias_cdmx.shp")
gdf_total_coyoacan = filter_coyoacan(gdf_total, alcaldias_shp)
print("Total de registros después de filtrar:", len(gdf_total_coyoacan))

# Paso 7: Limpiar geometrías
print("\nLimpiando geometrías...")
gdf_total_coyoacan = clean_geometries(gdf_total_coyoacan)
print("Total de registros después de limpiar geometrías:", len(gdf_total_coyoacan))

# Paso 8: Calcular área y densidad de población
print("\nCalculando área y densidad de población...")
gdf_total_coyoacan = calculate_area_density(gdf_total_coyoacan, 'pob_total', 'densidad_pob_total')
gdf_total_coyoacan = calculate_area_density(gdf_total_coyoacan, 'pob_hombres', 'densidad_hombres')
gdf_total_coyoacan = calculate_area_density(gdf_total_coyoacan, 'pob_mujeres', 'densidad_mujeres')

# Paso 9: Verificar áreas y filtrar geometrías pequeñas
print("\nEstadísticas de area_km2 antes de filtrar:")
print(gdf_total_coyoacan['area_km2'].describe())

area_threshold = 0.001  # Umbral mínimo de área en km²
gdf_total_coyoacan = gdf_total_coyoacan[gdf_total_coyoacan['area_km2'] > area_threshold]
print(f"\nTotal de registros después de filtrar por área mínima ({area_threshold} km²): {len(gdf_total_coyoacan)}")

# Recalcular densidades después de filtrar
print("\nRecalculando densidades después de filtrar por área mínima...")
gdf_total_coyoacan = calculate_area_density(gdf_total_coyoacan, 'pob_total', 'densidad_pob_total')
gdf_total_coyoacan = calculate_area_density(gdf_total_coyoacan, 'pob_hombres', 'densidad_hombres')
gdf_total_coyoacan = calculate_area_density(gdf_total_coyoacan, 'pob_mujeres', 'densidad_mujeres')

# Paso 10: Filtrar outliers en densidad
print("\nFiltrando outliers en densidad de población...")
densidad_threshold = gdf_total_coyoacan['densidad_pob_total'].quantile(0.99)
gdf_total_coyoacan = gdf_total_coyoacan[gdf_total_coyoacan['densidad_pob_total'] <= densidad_threshold]
print(f"Total de registros después de filtrar outliers de densidad: {len(gdf_total_coyoacan)}")

# Paso 11: Agregar el año al dataset
print("\nAgregando el año al dataset...")
gdf_total_coyoacan['anio'] = 2020

# Paso 12: Guardar el conjunto de datos limpio
print("\nGuardando el conjunto de datos limpio...")
output_path = os.path.join(BASE_PATH, "clean_data")
os.makedirs(output_path, exist_ok=True)
output_file = os.path.join(output_path, "coyoacan_poblacion_2020_clean.geojson")
gdf_total_coyoacan.to_file(output_file, driver='GeoJSON')
print(f"Conjunto de datos limpio guardado en {output_file}")

# Paso 13: Obtener valores máximos para la escala del mapa
max_density_total = gdf_total_coyoacan['densidad_pob_total'].quantile(0.95)
max_density_hombres = gdf_total_coyoacan['densidad_hombres'].quantile(0.95)
max_density_mujeres = gdf_total_coyoacan['densidad_mujeres'].quantile(0.95)

print(f"\nValor máximo para la escala del mapa de densidad total (percentil 95): {max_density_total}")
print(f"Valor máximo para la escala del mapa de densidad hombres (percentil 95): {max_density_hombres}")
print(f"Valor máximo para la escala del mapa de densidad mujeres (percentil 95): {max_density_mujeres}")

# Paso 14: Crear mapas con el conjunto de datos limpio
print("\nCreando mapas...")
m_total = create_folium_map(
    gdf_total_coyoacan,
    'densidad_pob_total',
    'Densidad de Población Total (hab/km²)',
    'YlOrRd',
    'pob_total',
    max_density_total
)
m_hombres = create_folium_map(
    gdf_total_coyoacan,
    'densidad_hombres',
    'Densidad de Población Masculina (hab/km²)',
    'Blues',
    'pob_hombres',
    max_density_hombres
)
m_mujeres = create_folium_map(
    gdf_total_coyoacan,
    'densidad_mujeres',
    'Densidad de Población Femenina (hab/km²)',
    'Purples',
    'pob_mujeres',
    max_density_mujeres
)

Cargando datos demográficos...

Datos cargados desde /home/jazzzfm/Documents/CoyoacanDataAnalysis/data/demografico/2020/total.shp:
              alc             loc amb_loc           ageb  pob_total  p_3ymas  \
0  Álvaro Obregón  Álvaro Obregón  Urbana  0901000011716     7042.0   6811.0   
1  Álvaro Obregón  Álvaro Obregón  Urbana  0901000012150     4588.0   4487.0   
2  Álvaro Obregón  Álvaro Obregón  Urbana  0901000011133     2203.0   2179.0   
3  Álvaro Obregón  Álvaro Obregón  Urbana  0901000011307     2914.0   2859.0   
4  Álvaro Obregón  Álvaro Obregón  Urbana  0901000010281      237.0    234.0   

   p_12yms  p_nacoe  p_vivoe  p_hli  ...   t_hli_h   t_afrmx  t_p12ym_sl  \
0   5896.0   1420.0    134.0  170.0  ...  2.451916  0.908833   34.107870   
1   4062.0    672.0    142.0   26.0  ...  0.579452  1.307759   39.217134   
2   2025.0    473.0    119.0   25.0  ...  1.101423  1.497957   39.111111   
3   2628.0    657.0    175.0   86.0  ...  2.973068  0.754976   41.400304   
4    204

In [4]:
m_total

In [5]:
m_hombres

In [ ]:
# Paso 15: Guardar mapas
print("\nGuardando mapas...")
m_total.save("densidad_poblacion_total.html")
m_hombres.save("densidad_poblacion_hombres.html")
m_mujeres.save("densidad_poblacion_mujeres.html")

print("\nMapas guardados exitosamente.")

In [87]:
import geopandas as gpd

BASE_PATH = os.path.expanduser("~/Documentos/CoyoacanDataAnalysis/data")

# Cargar shapefiles de AGEBs y colonias
ageb_shp = BASE_PATH + '/ageb_cdmx/poligono_ageb_urbanas_cdmx.shp'
colonias_shp = BASE_PATH + '/colonias/colonias_seduvi.shp'

In [88]:
# Asegurar que ambos GeoDataFrames tengan el mismo sistema de coordenadas (CRS)
gdf_colonias = gdf_colonias.to_crs(gdf_ageb.crs)

In [123]:
gdf_colonias[gdf_colonias['']]

,cve_ent,alcaldia,cve_col,colonia,pob_2010,area,perimetro,geometry
0,09,AZCAPOTZALCO,02-001,AGUILERA,2014.0,103907.260,1630.955,"POLYGON ((-99.15901 19.47374, -99.15882 19.473..."
1,09,AZCAPOTZALCO,02-002,ALDANA,3378.0,154934.607,1849.953,"POLYGON ((-99.14858 19.47156, -99.14863 19.471..."
2,09,AZCAPOTZALCO,02-005,ANGEL ZIMBRON,2737.0,242423.457,2189.321,"POLYGON ((-99.18794 19.46458, -99.188 19.46381..."
3,09,AZCAPOTZALCO,02-006,ARENAL,4817.0,286301.866,2155.264,"POLYGON ((-99.15187 19.46803, -99.15186 19.468..."
4,09,AZCAPOTZALCO,02-007,CENTRO DE AZCAPOTZALCO,3043.0,424806.534,3066.308,"POLYGON ((-99.18959 19.48519, -99.18946 19.485..."
...,...,...,...,...,...,...,...,...
1809,09,ALVARO OBREGON,10-236,VILLA PROGRESISTA,3089.0,94096.228,1308.859,"POLYGON ((-99.24369 19.35705, -99.24358 19.357..."
1810,09,ALVARO OBREGON,10-237,VILLA SOLIDARIDAD,588.0,30119.193,1108.924,"POLYGON ((-99.21851 19.38293, -99.21861 19.382..."
1811,09,ALVARO OBREGON,10-238,VILLA VERDUN,3324.0,641829.678,5955.574,"POLYGON ((-99.25629 19.34239, -99.25639 19.342..."
1812,09,ALVARO OBREGON,10-239,ZENON DELGADO,2658.0,107535.371,1919.855,"POLYGON ((-99.21591 19.3924, -99.21585 19.3922..."


In [90]:
# Realizar la unión espacial utilizando 'predicate' en lugar de 'op'
gdf_combined = gpd.sjoin(gdf_ageb, gdf_colonias, how='left', predicate='intersects')

In [91]:
print(gdf_combined.columns)

Index(['CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_AGEB', 'geometry',
       'index_right', 'cve_ent', 'alcaldia', 'cve_col', 'colonia', 'pob_2010',
       'area', 'perimetro'],
      dtype='object')


In [92]:
print(gdf_combined[['CVE_AGEB', 'colonia']].head())

  CVE_AGEB                      colonia
0     1716  SAN BERNABE OCOTEPEC (PBLO)
0     1716              LOMAS DE LA ERA
0     1716                   LAS CRUCES
0     1716                    EL TANQUE
0     1716          LOMAS DE LOS CEDROS


In [93]:
# Agrupando colonias por AGEB
gdf_ageb_colonias = gdf_combined.groupby('CVE_AGEB').agg({
    'colonia': lambda x: list(x.unique()),
    'geometry': 'first'  # Mantener una geometría por AGEB
}).reset_index()

In [95]:
# Convertir gdf_ageb_colonias a GeoDataFrame
gdf_ageb_colonias = gpd.GeoDataFrame(gdf_ageb_colonias, geometry='geometry')

# Añadir columna de año
gdf_ageb_colonias['year'] = 2020

# Guardar el archivo como shapefile
output_path = '~/Documentos/CoyoacanDataAnalysis/data/ageb_cdmx/ageb_colonias_2020.shp'
gdf_ageb_colonias.to_file(output_path)

/home/jazzzfm/Documentos/CoyoacanDataAnalysis/coyo/lib/python3.11/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
/home/jazzzfm/Documentos/CoyoacanDataAnalysis/coyo/lib/python3.11/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '['SANTA MARIA TEPEPAN (PBLO)', 'BOSQUE RESIDENCIAL DEL SUR (FRACC)', 'SAN BARTOLO EL CHICO', 'AMSA', 'SAN JUAN TEPEPAN', 'VALLE DE TEPEPAN', 'VALLE ESCONDIDO', 'ARENAL PUERTA TEPEPAN', 'ARENAL GUADALUPE TLALPAN', 'INDEPENDENCIA SAN RAMON (U HAB)', 'SAN JERONIMO ACULCO - LIDICE (PBLO)', 'INDEPENDENCIA BATAN SUR (U HAB)', 'INDEPENDENCIA BATAN NORTE (U HAB)', 'MIGUEL HIDALGO', 'BATAN VIEJO (EL MAESTRO)']' of field colonia has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


In [97]:
gdf_ageb_colonias

,CVE_AGEB,colonia,geometry,year
0,0010,"[EL ROSARIO A (U HAB), EL ROSARIO B (U HAB)]","POLYGON ((-99.20294 19.51461, -99.20307 19.514...",2020
1,0011,"[NARVARTE I, NARVARTE III, NARVARTE II, DEL VA...","POLYGON ((-99.1556 19.40172, -99.15641 19.4016...",2020
2,0012,"[COVE, AMERICA, DANIEL GARZA, ACUEDUCTO, ESTAD...","POLYGON ((-99.19757 19.40068, -99.19805 19.400...",2020
3,0013,"[POPULAR RASTRO, NICOLAS BRAVO, FELIPE PESCADO...","POLYGON ((-99.11897 19.45868, -99.1185 19.4585...",2020
4,0014,"[SANTA MARIA TEPEPAN (PBLO), BOSQUE RESIDENCIA...","POLYGON ((-99.1389 19.28224, -99.13873 19.2821...",2020
...,...,...,...,...
1947,5893,"[GRANJAS ESTRELLA I, SAN JUAN XALPA II, SAN NI...","POLYGON ((-99.08661 19.33545, -99.08662 19.335...",2020
1948,5906,"[PUENTE BLANCO, PRESIDENTES DE MEXICO, CASA BL...","POLYGON ((-99.06338 19.33353, -99.06212 19.332...",2020
1949,5910,"[PUENTE BLANCO, PRESIDENTES DE MEXICO]","POLYGON ((-99.06264 19.32828, -99.06267 19.328...",2020
1950,5925,"[GRANJAS ESTRELLA III, SAN NICOLAS TOLENTINO I...","POLYGON ((-99.08335 19.32944, -99.08238 19.329...",2020


In [101]:
import geopandas as gpd

# Cargar y filtrar los límites de Coyoacán
alcaldias_shp = f"{base_path}/limites/poligonos_alcaldias_cdmx.shp"
gdf_alcaldias = gpd.read_file(alcaldias_shp)
gdf_coyoacan = gdf_alcaldias[gdf_alcaldias['NOMGEO'] == 'Coyoacán']

# Verifica el CRS de gdf_coyoacan
print("CRS de Coyoacán:", gdf_coyoacan.crs)

# Asigna un CRS a gdf_ageb_colonias si no tiene uno
if gdf_ageb_colonias.crs is None:
    gdf_ageb_colonias.set_crs("EPSG:4326", inplace=True)  # Puedes cambiar EPSG:4326 según tus datos

# Asegúrate de que ambos GeoDataFrames tengan el mismo CRS
gdf_ageb_colonias = gdf_ageb_colonias.to_crs(gdf_coyoacan.crs)

# Realizar la unión espacial para asignar solo las colonias dentro de Coyoacán
gdf_coyoacan_colonias = gpd.sjoin(gdf_ageb_colonias, gdf_coyoacan, how='inner', predicate='within')

# Verifica los resultados
print(gdf_coyoacan_colonias[['CVE_AGEB', 'colonia']].head())


CRS de Coyoacán: EPSG:4326
   CVE_AGEB                                            colonia
7      0018  [DEL CARMEN, XOCO, SANTA CATARINA (BARR), HACI...
12     0022                                 [DEL CARMEN, XOCO]
27     0037                                       [DEL CARMEN]
48     0056  [PARQUE SAN ANDRES, SAN MATEO (BARR), SAN LUCA...
65     0075  [CAMPESTRE CHURUBUSCO, COUNTRY CLUB, PARQUE SA...


In [122]:
import pandas as pd

# Asegúrate de que la columna 'colonia' esté en formato lista si no lo está ya
# gdf_coyoacan_colonias['colonia'] = gdf_coyoacan_colonias['colonia'].apply(lambda x: x if isinstance(x, list) else [x])

# Expande las colonias, creando una fila por cada colonia en cada CVE_AGEB
gdf_expanded = gdf_coyoacan_colonias.explode('colonia').reset_index(drop=True)

# Visualiza el resultado
print("Dataframe expandido por colonia:")
print(gdf_expanded[['CVE_AGEB','CVEGEO', 'NOMGEO', 'colonia']].head())


Dataframe expandido por colonia:
  CVE_AGEB CVEGEO    NOMGEO                            colonia
0     0018  09003  Coyoacán                         DEL CARMEN
1     0018  09003  Coyoacán                               XOCO
2     0018  09003  Coyoacán              SANTA CATARINA (BARR)
3     0018  09003  Coyoacán  HACIENDA DE GUADALUPE CHIMALISTAC
4     0018  09003  Coyoacán                            FLORIDA


In [110]:
print("Columnas de gdf_expanded:", gdf_expanded.columns)
print("Muestra de valores en gdf_expanded 'CVE_AGEB':", gdf_expanded['CVE_AGEB'].unique()[:5])
# Filtrar solo registros de Coyoacán si hay una columna de nombre de alcaldía
if 'NOMGEO' in gdf_expanded.columns:
    gdf_coyoacan_expanded = gdf_expanded[gdf_expanded['NOMGEO'] == 'Coyoacán']
    print("Cantidad de registros en Coyoacán:", len(gdf_coyoacan_expanded))
else:
    print("La columna con el nombre de la alcaldía no está disponible.")
print("CRS de gdf_expanded:", gdf_expanded.crs)
print("CRS de gdf_total_coyoacan:", gdf_total_coyoacan.crs)

Columnas de gdf_expanded: Index(['CVE_AGEB', 'colonia', 'geometry', 'year', 'index_right', 'CVEGEO',
       'CVE_ENT', 'CVE_MUN', 'NOMGEO'],
      dtype='object')
Muestra de valores en gdf_expanded 'CVE_AGEB': ['0018' '0022' '0037' '0056' '0075']
Cantidad de registros en Coyoacán: 696
CRS de gdf_expanded: EPSG:4326
CRS de gdf_total_coyoacan: EPSG:32614


In [112]:
# Verifica las columnas en gdf_total_coyoacan
print("Columnas en gdf_total_coyoacan:", gdf_total_coyoacan.columns)

Columnas en gdf_total_coyoacan: Index(['alc', 'loc', 'amb_loc', 'CVE_AGEB', 'pob_total', 'p_3ymas', 'p_12yms',
       'p_nacoe', 'p_vivoe', 'p_hli', 'p_hl_nh', 'p_hli_h', 'p_afrmx',
       'p_p12ym_sl', 'p_p12ym_c', 'p_p12ym_sp', 'p_catlc', 'p_criev',
       'p_trsrl', 'p_sinrl', 't_nacoe', 't_vivoe', 't_hli', 't_hl_nh',
       't_hli_h', 't_afrmx', 't_p12ym_sl', 't_p12ym_c', 't_p12ym_sp',
       't_catlc', 't_criev', 't_trsrl', 't_sinrl', 'geometry', 'pob_hombres',
       'pob_mujeres', 'area_km2', 'densidad_pob_total', 'densidad_hombres',
       'densidad_mujeres', 'anio'],
      dtype='object')


In [118]:
# Normalizar la longitud de 'CVE_AGEB' en ambos DataFrames a 4 caracteres (o el valor que corresponda)
gdf_expanded['CVE_AGEB'] = gdf_expanded['CVE_AGEB'].astype(str).str.zfill(4)
gdf_total_coyoacan['CVE_AGEB'] = gdf_total_coyoacan['CVE_AGEB'].astype(str).str.zfill(4)

# Verificar algunos valores para confirmar la normalización
print("Valores únicos en gdf_expanded después de la normalización:")
print(gdf_expanded['CVE_AGEB'].unique()[:5])

print("Valores únicos en gdf_total_coyoacan después de la normalización:")
print(gdf_total_coyoacan['CVE_AGEB'].unique()[:5])

# Intentar la unión de nuevo
gdf_final = gdf_expanded.merge(
    gdf_total_coyoacan,
    on='CVE_AGEB',  
    how='left'
)

# Verificar los resultados
print("Dataframe final con datos demográficos por colonia:")
print(gdf_final[['CVE_AGEB', 'colonia', 'pob_total', 'densidad_pob_total']].head())

Valores únicos en gdf_expanded después de la normalización:
['0018' '0022' '0037' '0056' '0075']
Valores únicos en gdf_total_coyoacan después de la normalización:
['1514' '1497' '1463' '1482' '1073']
Dataframe final con datos demográficos por colonia:
  CVE_AGEB                            colonia  pob_total  densidad_pob_total
0     0018                         DEL CARMEN     1658.0         2387.963626
1     0018                               XOCO     1658.0         2387.963626
2     0018              SANTA CATARINA (BARR)     1658.0         2387.963626
3     0018  HACIENDA DE GUADALUPE CHIMALISTAC     1658.0         2387.963626
4     0018                            FLORIDA     1658.0         2387.963626


In [119]:
# Extraer los últimos 4 dígitos de 'CVE_AGEB' en gdf_total_coyoacan
gdf_total_coyoacan['CVE_AGEB'] = gdf_total_coyoacan['CVE_AGEB'].astype(str).str[-4:]

# Verificar algunos valores para asegurarse de que la extracción fue exitosa
print("Valores únicos en gdf_total_coyoacan después de la extracción de los últimos 4 dígitos:")
print(gdf_total_coyoacan['CVE_AGEB'].unique()[:5])

# Intentar la unión de nuevo
gdf_final = gdf_expanded.merge(
    gdf_total_coyoacan,
    on='CVE_AGEB',  
    how='left'
)

# Verificar los resultados
print("Dataframe final con datos demográficos por colonia:")
print(gdf_final[['CVE_AGEB', 'colonia', 'pob_total', 'densidad_pob_total']].head())


Valores únicos en gdf_total_coyoacan después de la extracción de los últimos 4 dígitos:
['1514' '1497' '1463' '1482' '1073']
Dataframe final con datos demográficos por colonia:
  CVE_AGEB                            colonia  pob_total  densidad_pob_total
0     0018                         DEL CARMEN     1658.0         2387.963626
1     0018                               XOCO     1658.0         2387.963626
2     0018              SANTA CATARINA (BARR)     1658.0         2387.963626
3     0018  HACIENDA DE GUADALUPE CHIMALISTAC     1658.0         2387.963626
4     0018                            FLORIDA     1658.0         2387.963626
